In [1]:
import pandas as pd
import numpy as np

In [2]:
def checkSomething(originalColumn, possibleDuplicates):
    for uniqueValue in originalColumn.unique():
        if len(possibleDuplicates.loc[originalColumn == uniqueValue].unique()) > 1:
            print(uniqueValue, ':', possibleDuplicates.loc[originalColumn == uniqueValue].unique())

In [3]:
df = pd.read_csv('line_score.csv')

In [4]:
df.columns

Index(['game_date_est', 'game_sequence', 'game_id', 'team_id_home',
       'team_abbreviation_home', 'team_city_name_home', 'team_nickname_home',
       'team_wins_losses_home', 'pts_qtr1_home', 'pts_qtr2_home',
       'pts_qtr3_home', 'pts_qtr4_home', 'pts_ot1_home', 'pts_ot2_home',
       'pts_ot3_home', 'pts_ot4_home', 'pts_ot5_home', 'pts_ot6_home',
       'pts_ot7_home', 'pts_ot8_home', 'pts_ot9_home', 'pts_ot10_home',
       'pts_home', 'team_id_away', 'team_abbreviation_away',
       'team_city_name_away', 'team_nickname_away', 'team_wins_losses_away',
       'pts_qtr1_away', 'pts_qtr2_away', 'pts_qtr3_away', 'pts_qtr4_away',
       'pts_ot1_away', 'pts_ot2_away', 'pts_ot3_away', 'pts_ot4_away',
       'pts_ot5_away', 'pts_ot6_away', 'pts_ot7_away', 'pts_ot8_away',
       'pts_ot9_away', 'pts_ot10_away', 'pts_away'],
      dtype='object')

In [5]:
df.head()

,game_date_est,game_sequence,game_id,team_id_home,team_abbreviation_home,team_city_name_home,team_nickname_home,team_wins_losses_home,pts_qtr1_home,pts_qtr2_home,...,pts_ot2_away,pts_ot3_away,pts_ot4_away,pts_ot5_away,pts_ot6_away,pts_ot7_away,pts_ot8_away,pts_ot9_away,pts_ot10_away,pts_away
0,1946-11-01 00:00:00,NaN,24600001,1610610035,HUS,Toronto,Huskies,-,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68
1,1946-11-02 00:00:00,NaN,24600003,1610610034,BOM,St. Louis,Bombers,-,16.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51
2,1946-11-02 00:00:00,NaN,24600002,1610612738,BOS,Boston,Celtics,-,10.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59
3,1946-11-02 00:00:00,NaN,24600004,1610610025,CHS,Chicago,Stags,-,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47
4,1946-11-02 00:00:00,NaN,24600005,1610610036,WAS,Washington,Capitols,-,21.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33


## Remove all games with OT and OT data

In [6]:
df['pts_ot1_home'].unique()

array([18., nan,  7.,  4., 10., 15., 12.,  3., 13.,  6.,  5.,  8., 20.,
        9., 11., 14.,  2., 17.,  0.,  1., 16., 19., 21., 22., 24., 23.,
       25.])

In [7]:
print(sum(df['pts_ot1_home'].isnull()))
print(sum(df['pts_ot1_home'] == 0))

29428
27971


In [8]:
df = df.loc[df['pts_ot1_home'].isin([np.nan, 0])]
df.drop(['pts_ot1_home', 'pts_ot2_home',
       'pts_ot3_home', 'pts_ot4_home', 'pts_ot5_home', 'pts_ot6_home',
       'pts_ot7_home', 'pts_ot8_home', 'pts_ot9_home', 'pts_ot10_home', 'pts_ot1_away', 'pts_ot2_away', 'pts_ot3_away', 'pts_ot4_away',
       'pts_ot5_away', 'pts_ot6_away', 'pts_ot7_away', 'pts_ot8_away',
       'pts_ot9_away', 'pts_ot10_away'], axis = 1, inplace = True);

# Normalize Data

## 1. PK: Abbreviation

### Eliminate Duplicate City Names

In [9]:
checkSomething(df['team_abbreviation_home'], df['team_city_name_home']) 

KCK : ['Kansas City-Omaha' 'Kansas City']
LAC : ['Los Angeles' 'LA']
NOK : ['New Orleans' 'New Orleans/Oklahoma City']


In [10]:
df['team_nickname_home'].loc[df['team_abbreviation_home'] == 'NOK'].unique()

array(['Hornets'], dtype=object)

In [11]:
df.replace(['Kansas City-Omaha', 'LA'], ['Kansas City' , 'Los Angeles'], inplace = True)

df.loc[df['team_abbreviation_home'] == 'NOK', 'team_city_name_home'] = 'New Orleans/Oklahoma City'
df.loc[df['team_abbreviation_away'] == 'NOK', 'team_city_name_away'] = 'New Orleans/Oklahoma City'

In [12]:
checkSomething(df['team_abbreviation_home'], df['team_city_name_home'])
checkSomething(df['team_abbreviation_away'], df['team_city_name_away'])

### Eliminate Duplicate IDs

In [13]:
checkSomething(df['team_abbreviation_home'], df['team_id_home'])

WAS : [1610610036 1610612764]


In [14]:
print(df['team_abbreviation_home'].loc[df['team_id_home'] == 1610610036].unique())
print(df['team_abbreviation_home'].loc[df['team_id_home'] == 1610612764].unique())

['WAS']
['CHP' 'CHZ' 'BLT' 'CAP' 'WAS']


In [15]:
df.loc[df['team_abbreviation_home'] == 'WAS', 'team_id_home'] = 1610610036
df.loc[df['team_abbreviation_away'] == 'WAS', 'team_id_away'] = 1610610036

In [16]:
checkSomething(df['team_abbreviation_home'], df['team_id_home'])
checkSomething(df['team_abbreviation_away'], df['team_id_away'])

In [17]:
checkSomething(df['team_abbreviation_home'], df['team_city_name_home'])
checkSomething(df['team_abbreviation_away'], df['team_city_name_away'])

### Create Table

In [18]:
dfTeamInfo = df[['team_abbreviation_home', 'team_city_name_home', 'team_id_home']]
dfTeamInfo = dfTeamInfo.rename(columns = {'team_abbreviation_home': 'Abbreviation', 'team_city_name_home': 'City Name', 'team_id_home': 'ID'})
dfTeamInfo.head()

,Abbreviation,City Name,ID
1,BOM,St. Louis,1610610034
2,BOS,Boston,1610612738
3,CHS,Chicago,1610610025
4,WAS,Washington,1610610036
5,HUS,Toronto,1610610035


In [19]:
dfTeamInfo = dfTeamInfo.drop_duplicates(subset='Abbreviation', keep="first").reset_index(drop = True)
df.drop(['team_id_home', 'team_city_name_home', 'team_nickname_home', 'team_id_away', 'team_city_name_away', 'team_nickname_away'],
        axis = 1, inplace = True)
dfTeamInfo

,Abbreviation,City Name,ID
0,BOM,St. Louis,1610610034
1,BOS,Boston,1610612738
2,CHS,Chicago,1610610025
3,WAS,Washington,1610610036
4,HUS,Toronto,1610610035
...,...,...,...
70,NOK,New Orleans/Oklahoma City,1610612740
71,PHO,Phoenix,1610612756
72,OKC,Oklahoma City,1610612760
73,BKN,Brooklyn,1610612751


## 2. PK: Game ID

In [20]:
df['game_id'].duplicated().sum()

0

In [21]:
df.columns

Index(['game_date_est', 'game_sequence', 'game_id', 'team_abbreviation_home',
       'team_wins_losses_home', 'pts_qtr1_home', 'pts_qtr2_home',
       'pts_qtr3_home', 'pts_qtr4_home', 'pts_home', 'team_abbreviation_away',
       'team_wins_losses_away', 'pts_qtr1_away', 'pts_qtr2_away',
       'pts_qtr3_away', 'pts_qtr4_away', 'pts_away'],
      dtype='object')

In [22]:
dfGameInfo = df[['game_id', 'game_date_est', 'game_sequence']]
pd.to_datetime(dfGameInfo['game_date_est']).dt.date

df.drop(['game_date_est','game_sequence'], axis = 1, inplace = True)

### 3. PK: GameID, Home Abbreviation

In [23]:
subsetHome = ['game_id',
          'team_abbreviation_home',
          'team_wins_losses_home',
          'pts_qtr1_home',
          'pts_qtr2_home',
          'pts_qtr3_home',
          'pts_qtr4_home',
          'pts_home']

dfHomeGames = df[subsetHome]

df.drop(['team_wins_losses_home',
          'pts_qtr1_home',
          'pts_qtr2_home',
          'pts_qtr3_home',
          'pts_qtr4_home',
          'pts_home'],
        axis = 1, inplace = True)

### 4. PK: GameID, Away Abbreviation

In [24]:
subsetAway = ['game_id',
          'team_abbreviation_away',
          'team_wins_losses_away',
          'pts_qtr1_away',
          'pts_qtr2_away',
          'pts_qtr3_away',
          'pts_qtr4_away',
          'pts_away']

dfAwayGames = df[subsetAway]

df.drop(['team_wins_losses_away',
          'pts_qtr1_away',
          'pts_qtr2_away',
          'pts_qtr3_away',
          'pts_qtr4_away',
          'pts_away'],
        axis = 1, inplace = True)

In [25]:
pd.set_option('expand_frame_repr', False)

print('Team Information')
print(dfTeamInfo.head())
print()
print('Game MetaData')
print(dfGameInfo.head())
print()
print('Home Team Game Data')
print(dfHomeGames.head())
print()
print('Away Team Game Data')
print(dfAwayGames.head())

Team Information
  Abbreviation   City Name          ID
0          BOM   St. Louis  1610610034
1          BOS      Boston  1610612738
2          CHS     Chicago  1610610025
3          WAS  Washington  1610610036
4          HUS     Toronto  1610610035

Game MetaData
    game_id        game_date_est  game_sequence
1  24600003  1946-11-02 00:00:00            NaN
2  24600002  1946-11-02 00:00:00            NaN
3  24600004  1946-11-02 00:00:00            NaN
4  24600005  1946-11-02 00:00:00            NaN
5  24600006  1946-11-03 00:00:00            NaN

Home Team Game Data
    game_id team_abbreviation_home team_wins_losses_home  pts_qtr1_home  pts_qtr2_home  pts_qtr3_home  pts_qtr4_home  pts_home
1  24600003                    BOM                     -           16.0           16.0           18.0            6.0        56
2  24600002                    BOS                     -           10.0           16.0           14.0           13.0        53
3  24600004                    CHS          